In [1]:
%%time
import geopandas as gpd
import shapely
import pandas as pd
import numpy as np
import re
from pygeodesy.ellipsoidalVincenty import LatLon
from pygeodesy import Datums



# ================================================ #
#     Считаем длину входных линий по эллипсоиду    #
# ================================================ #



# Считываем входной файл и генерируем dataframe
# df = gpd.read_file("ellipsoid_distance_ref.geojson")
df = gpd.read_file("m12_roadway.geojson")

# Создаем рабочий dataframe и добавляем столбец с расстоянием
calculated_df = df.assign(ellipsoid_distance=0)

# Запускаем цикл подсчета расстояния для всех значений таблицы
for required_id in range(len(df)):
    if 'MULTILINESTRING' in str(df['geometry'][required_id]):
        coords_pos_start = str(df['geometry'][required_id]).find('((') + 2
    elif 'LINESTRING' in str(df['geometry'][required_id]):
        coords_pos_start = str(df['geometry'][required_id]).find('(') + 1
    # Фильтруем мусор
    initial_string = str(df['geometry'][required_id])[coords_pos_start::]
    filtered_string = re.sub(r"[)(]", "", initial_string)
    # Разделяем WKT по вершинкам LatLon
    coords_list = filtered_string.split(', ')
    # Переводим всё в нумпай массив
    initial_array = np.array(coords_list, str)
    coords_pairs = []
    # Запускаем цикл по сплиту и генерации пар
    for i in range(len(initial_array)):
        coords_pairs.append(initial_array[i].split(' '))
    coords_array = np.array(coords_pairs, float)
    distance_list = []
    # Запускаем цикл по расчету расстояния по эллипсоиду
    for index in range(len(coords_array)):
        if index+1 < len(coords_array):
            first_point = LatLon(coords_array[index][1], coords_array[index][0], datum=Datums.WGS84)
            second_point = LatLon(coords_array[index+1][1], coords_array[index+1][0], datum=Datums.WGS84)
            distance_list.append(first_point.distanceTo(second_point))
    # Засовываем всё в нумпай массив
    distance_array = np.array(distance_list, float)
    # Добавляем в рабочий df рассчитанное расстояние
    calculated_df.at[required_id, 'ellipsoid_distance'] = sum(distance_array)


# Экспортируем полученные данные в нужный формат
export_gdf = gpd.GeoDataFrame(calculated_df, geometry="geometry")
# export_gdf.to_file("calculated_ellipsoid_distance.geojson", driver="GeoJSON")



# =============================================== #
#  Генерируем точки каждые X метров по эллипсоиду #
# =============================================== #



required_distance = 1000 # метры

step_array = np.arange(0, 1, 1/(export_gdf['ellipsoid_distance'][0]/required_distance))

"""
    
    ОПЦИОНАЛЬНО -- ГАЛОЧКА, ЕСЛИ НУЖНА КОНЕЧНАЯ ТОЧКА:
                ДОБАВИТЬ СЛЕДУЮЩУЮ СТРОКУ
                
    ==================================================
            step_array = np.append(step_array, 1)
    ==================================================
    
"""
# ==================================== #
#         Опциональная функция         #
step_array = np.append(step_array, 1)
# ==================================== #

# Параметр normalized=True означает, что это доля прямой, а не метр
parts_list = shapely.line_interpolate_point(export_gdf['geometry'][0], step_array, normalized=True).tolist()

temp_df = pd.DataFrame(parts_list, columns=['geometry'])
temp_df['id'] = np.arange(1, 1+len(temp_df), 1)
temp_df['distance'] = np.arange(0, required_distance*len(temp_df), required_distance)
temp_df = temp_df.replace(temp_df['distance'][len(temp_df)-1],export_gdf['ellipsoid_distance'][0])

parts_gdf = gpd.GeoDataFrame(temp_df, geometry="geometry")
# parts_gdf.to_file("test.geojson", driver="GeoJSON")

/home/sda/.local/lib/python3.10/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


CPU times: user 3.62 s, sys: 2.14 s, total: 5.76 s
Wall time: 2.85 s


<timed exec>:54: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '416536.46631904336' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
